## Building the Augmented Data set
- By exploiting the previously written code (in the trajectory generation notebook), we synthetically augment the dataset in order to have more working data.
- We begin by loading medoids and getting/serializing graphs on disk.

In [14]:
#imports cell

import pandas as pd
import matplotlib.pyplot as plt
import osmnx as ox 
from sklearn.neighbors import KDTree
import networkx as nx
import folium
import random
from geographiclib.geodesic import Geodesic
import math
import yaml

with open("conf.yaml") as f:
    conf = yaml.load(f, Loader = yaml.FullLoader)

#base_path = conf["base_path"]
data_path = conf["data_path"]

In [1]:


#in order to load medoids only once
medoids = pd.read_csv(data_path + "augmented_medoids.csv", usecols = ["lat", "lon"])
print(medoids.head())

n_med = len(medoids)
print("We have a total of {:d} starting/arrival points".format(n_med))

         lat         lon
0  39.856215  116.640477
1  39.652854  115.846842
2  39.944152  116.721681
3  39.952524  116.549498
4  39.832093  116.079627
We have a total of 181 starting/arrival points


#### We download graphs for each network type and serialize them on disk in order to save us a lot of time during the trajectory generation.

In [ ]:
%%time
#get graphs from place and serialize them on disk

#get the graphs
D = ox.graph_from_place('Beijing, China', which_result=2, network_type='drive')
B = ox.graph_from_place('Beijing, China', which_result=2, network_type='bike')
W = ox.graph_from_place('Beijing, China', which_result=2, network_type='walk')

In [ ]:
#serialize them on disk
ox.save_graphml(D, data_path+"drive_graph.graphml")
ox.save_graphml(B, data_path+"bike_graph.graphml")
ox.save_graphml(W, data_path+"walk_graph.graphml")

print("Serialized graphs on disk")

In [3]:
#Load graphs from files if already downloaded

D = ox.load_graphml(data_path+"drive_graph.graphml")
B = ox.load_graphml(data_path+"bike_graph.graphml")
W = ox.load_graphml(data_path+"walk_graph.graphml")


#### We use the put_datetime helper function in order to put datetimes in the generated trajectories.

In [15]:
import random
import time

def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d %H:%M:%S', prop)

#print(random_date("2008-06-01 00:00:00", "2008-08-31 23:59:00", random.random()))

def put_datetime(trajs_df, speed):
    
    from geopy.distance import geodesic
    import datetime
    from datetime import timedelta


    traj_copy = trajs_df.copy(deep = True)
    traj_copy["date_time"] = "2008-06-01 00:00:00"

    #selecting each traj by uid and tid
    
    overlaps = 0
    index = 0
    
    for uid in range(traj_copy.uid.min(), traj_copy.uid.max() + 1):
        user = traj_copy[traj_copy["uid"] == uid]
        
        #reset intervals
        intervals = []
        
        tid = user.tid.min()
        max_tid = user.tid.max()
        
        
        
        while (tid <= max_tid):
            
            initial_tid_index = index

            #get a random date in our range
            date = random_date("2008-06-01 00:00:00", "2008-08-31 23:59:00", random.random())
            date_time_obj = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

            #print("Trajectory {:d} starting at {:s}".format(tid, date))
            traj = user[user["tid"] == tid]
            #put the starting date at the beginning of the traj
            traj_copy.iat[index, 4] = date
            t_since_start = 0
            d_tot = 0
            #in order to do it only once
            t_len = len(traj)

            for i in range(1, t_len):
                try : 
                    #print(traj.iloc[i])
                    dist = geodesic((traj.iloc[i-1].lat, traj.iloc[i].lon), \
                                               (traj.iloc[i].lat, traj.iloc[i].lon))

                    d_tot += dist.meters

                    #gets m / m/s
                    tdelta = (dist.meters)/speed
                    t_since_start += tdelta

                    row_dt = date_time_obj + timedelta(seconds = t_since_start)
                    index += 1
                    traj_copy.iat[index, 4] =  row_dt

                except IndexError:
                    print(i)
                    #print(traj.iloc[i-1])
                    #print(traj.iloc[i])
                    
            n_interval = pd.Interval(date_time_obj.timestamp(), row_dt.timestamp())
            
            #check if the trajectory is time-overlapping with another one
            overlapping = False #stupid flag
            for interval in intervals:
                if (interval.overlaps(n_interval)):
                    overlapping = True
                    overlaps += 1
                    break
                    
            if (overlapping):
                index = initial_tid_index
                continue
                
            else:
                intervals.append(n_interval) 
                tid += 1
                index += 1
                
            #print("Total time for traj n {:d}: {:f} minutes. meters: {:f}".format(tid, t_since_start/60, d_tot))
            #print(traj.iloc[0].date_time)
            #print(traj.iloc[i].date_time)

            #print(traj)
    print("Put datetimes on {:d} rows, generated {:d} overlapping trajectories".format(index - 1, overlaps))
    return traj_copy

#### We define the interpolation and the faker functions to be called for each generation.

In [16]:
#interpolates the dataframe
def interpolator(gdf, uid):
    
    cols = ["lat", "lon", "uid", "tid"]
    #traj_df = pd.DataFrame(columns = cols)
    rows = []
    
    #meters for interpolation
    k = 5
    geod = Geodesic.WGS84

    for i in range(len(gdf) - 1):

        l = geod.InverseLine(gdf.iloc[i].y, gdf.iloc[i].x, gdf.iloc[i+1].y, gdf.iloc[i+1].x)
        ds = k; n = int(math.ceil(l.s13 / ds))
        for i in range(n + 1):
            #if i == 0:
                #print( "distance latitude longitude azimuth")
            s = min(ds * i, l.s13)
            g = l.Position(s, Geodesic.STANDARD | Geodesic.LONG_UNROLL)
            lat = g["lat2"]
            lon = g["lon2"]
            
            new_row = [lat, lon, uid, 0]
            rows.append(new_row)
            #print(traj_df.head())
    
    traj_df = pd.DataFrame(rows, columns = cols)
    
    return traj_df
    



#faker function
def faker(profile, n_users):
    #traj faker helper fun
    
    cols = ["lat", "lon", "uid", "tid"]
    fake_trajs = pd.DataFrame(columns = cols)
    #uid to recognize different types of fake users
    fake_uid = 0
    speed = 0
    
    if (profile == "drive"):
        G = ox.load_graphml(data_path+"drive_graph.graphml")
        fake_uid = 1000
        speed = 11.1
        
    elif (profile == "bike"):
        G = ox.load_graphml(data_path+"bike_graph.graphml")
        fake_uid = 2000
        speed = 4.3

    elif (profile == "walk"):
        G = ox.load_graphml(data_path+"walk_graph.graphml")
        fake_uid = 3000
        speed = 1.38

    else:
        print("not a valid profile!")
        return None
    
    print("Loaded {:s} graph from disk".format(profile))
    
    
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
    #print(gdf_nodes)
    tree = KDTree(gdf_nodes[['y', 'x']], metric='euclidean')
    
    n_trajs = 0
    errs = 0
    trajs = []
    
    for i in range(0, n_users):
        for j in range(5, random.randint(10, 15)):

            #sample 2 indexes
            picks = random.sample(range(0, n_med), 2)
            med_a = picks[0]
            med_b = picks[1]

            #get lat and lng for medoids
            med_a = (medoids.iloc[med_a].lat, medoids.iloc[med_a].lon)
            med_b = (medoids.iloc[med_b].lat, medoids.iloc[med_b].lon)

            #get the nearest points in the gdf
            med_a_idx = tree.query([med_a], k=1, return_distance=False)[0]
            med_b_idx = tree.query([med_b], k=1, return_distance=False)[0]

            closest_node_to_a = gdf_nodes.iloc[med_a_idx].index.values[0]
            closest_node_to_b = gdf_nodes.iloc[med_b_idx].index.values[0]  

            #calculate the shortest path
            try:
                path = nx.shortest_path(G, 
                             closest_node_to_a,
                             closest_node_to_b,
                             weight='length')
                n_trajs += 1

            #happens when there's not path between two points    
            except nx.NetworkXNoPath:
                errs += 1
                
                
            #print(path)
            gdf = gdf_nodes.loc[path]
            #print("Gdf number {:d}".format(n_trajs))
            #print(gdf.head())
            
            traj = interpolator(gdf, fake_uid)
            traj["tid"] = n_trajs
            
            trajs.append(traj)
            #fake_trajs = fake_trajs.append(traj, ignore_index = True)
            
            
            #print route for checking purposes
            """fig, ax = ox.plot_graph_route(G, path, fig_height=10, 
                                  fig_width=10, 
                                  show=False, close=False, 
                                  edge_color='black',
                                  orig_dest_node_color='green',
                                  route_color='green')
            plt.show()"""
            
        #on to another user!
        fake_uid += 1

    fake_trajs = pd.concat(trajs, ignore_index = True)
    print("generated {:d} trajectories for {:d} users with a {:s} profile. {:d} errors generated"
          .format(n_trajs, n_users, profile, errs))
    
    #print(fake_trajs.head())
    
    print("now putting datetimes on generated trajectories")
    return put_datetime(fake_trajs, speed)



#### We test our faker function with a relatively small generation

In [17]:
%%time
test = faker("drive", 2)

Loaded drive graph from disk
generated 11 trajectories for 2 users with a drive profile. 1 errors generated
now putting datetimes on generated trajectories
Put datetimes on 101005 rows, generated 0 overlapping trajectories
CPU times: user 2min 52s, sys: 5.93 s, total: 2min 58s
Wall time: 3min 18s


In [19]:
print(test)

              lat         lon   uid  tid                   date_time
0       39.978994  116.347534  1000    1         2008-06-24 03:48:48
1       39.978949  116.347536  1000    1  2008-06-24 03:48:48.450114
2       39.978904  116.347538  1000    1  2008-06-24 03:48:48.900228
3       39.978859  116.347540  1000    1  2008-06-24 03:48:49.350342
4       39.978814  116.347543  1000    1  2008-06-24 03:48:49.800456
...           ...         ...   ...  ...                         ...
101001  39.834279  116.602001  1001   11  2008-06-29 11:41:21.179171
101002  39.834324  116.602002  1001   11  2008-06-29 11:41:21.629614
101003  39.834369  116.602002  1001   11  2008-06-29 11:41:22.080058
101004  39.834414  116.602002  1001   11  2008-06-29 11:41:22.530501
101005  39.834452  116.602003  1001   11  2008-06-29 11:41:22.911635

[101006 rows x 5 columns]


#### Having succeeded in running our faker function, we proceed to the true dataset augmentation

In [20]:
%%time
num_cars = conf["cars"]
num_bikes = conf["bikes"]
num_pedestrians = conf["pedestrians"]

print("Now starting generation with {:d} cars, {:d} bikes and {:d} pedestrians"
     .format(num_cars, num_bikes, num_pedestrians))

cars = faker("drive", num_cars)
bikes = faker("bike", num_bikes)
pedestrians = faker("walk", num_pedestrians)

Now starting generation with 70 cars, 30 bikes and 100 pedestrians
Loaded drive graph from disk
generated 516 trajectories for 70 users with a drive profile. 9 errors generated
now putting datetimes on generated trajectories
Put datetimes on 3890916 rows, generated 0 overlapping trajectories
Loaded bike graph from disk
generated 219 trajectories for 30 users with a bike profile. 1 errors generated
now putting datetimes on generated trajectories
Put datetimes on 1602609 rows, generated 1 overlapping trajectories
Loaded walk graph from disk
generated 778 trajectories for 100 users with a walk profile. 0 errors generated
now putting datetimes on generated trajectories
Put datetimes on 5810613 rows, generated 10 overlapping trajectories
CPU times: user 4h 42min 54s, sys: 6min 3s, total: 4h 48min 58s
Wall time: 5h 3min 19s


In [21]:
%%time
cars.to_csv(data_path+"augmented_cars.csv")
bikes.to_csv(data_path+"augmented_bikes.csv")
pedestrians.to_csv(data_path+"augmented_pedestrians.csv")

CPU times: user 1min 51s, sys: 4.35 s, total: 1min 55s
Wall time: 2min 12s


In [13]:
print(bikes)

               lat         lon   uid  tid                   date_time
0        40.087859  116.294835  2000    1         2008-06-29 19:13:11
1        40.087820  116.294864  2000    1  2008-06-29 19:13:12.008406
2        40.087781  116.294893  2000    1  2008-06-29 19:13:13.016812
3        40.087742  116.294922  2000    1  2008-06-29 19:13:14.025218
4        40.087702  116.294952  2000    1  2008-06-29 19:13:15.033625
...            ...         ...   ...  ...                         ...
3179726  39.701495  116.719977  2049  389  2008-08-24 22:16:21.741904
3179727  39.701479  116.719922  2049  389  2008-08-24 22:16:22.154408
3179728  39.701463  116.719868  2049  389  2008-08-24 22:16:22.566914
3179729  39.701448  116.719813  2049  389  2008-08-24 22:16:22.979420
3179730  39.701439  116.719784  2049  389  2008-08-24 22:16:23.196714

[3179731 rows x 5 columns]


Let's see if the trajectory generation worked correctly

In [12]:
import skmob

#print(trajs)

tdf = skmob.TrajDataFrame(trajs[(trajs["uid"] == 1000)], longitude = "lon", datetime = "date_time")
print(tdf)


tdf.plot_trajectory(zoom=12, weight=3, opacity=0.9, tiles='Stamen Toner')

NameError: name 'trajs' is not defined

### We put everything together and serialize the resulting complete dataset on disk

In [22]:
col_names = ["lat", "lon", "uid", "tid", "date_time"] 

pedestrians = pd.read_csv(data_path + "augmented_pedestrians.csv", parse_dates = True, infer_datetime_format = True
                        ,index_col = 0)
bikes = pd.read_csv(data_path + "augmented_bikes.csv", parse_dates = True, infer_datetime_format = True
                        ,index_col = 0)
cars = pd.read_csv(data_path + "augmented_cars.csv", parse_dates = True, infer_datetime_format = True
                        ,index_col = 0)

cols = ["date_time", "lat", "lon", "tid", "uid"]

df = pd.read_csv(data_path + "complete_with_tids.csv", \
                 usecols = cols, parse_dates = True, infer_datetime_format = True)
#restricting to beijing area
df = df[(df['lat'].between(39.54, 40.3)) & (df['lon'].between(115.75, 117.13))]

#restricting to june - august 2008
start_time = "2008-06-01 00:00:00"
end_time = "2008-08-31 23:59:00"

original = (df[(df.date_time > start_time) & (df.date_time < end_time)]).copy()

print(original.head())
print(original.tid.max())

#augmented["date_time"] = pd.to_datetime(augmented["date_time"], format = "%Y-%m-%d %H:%M:%S.%f")

                   date_time        lat         lon   tid  uid
2101262  2008-06-17 09:44:44  39.976060  116.310953  1440   10
2101263  2008-06-17 09:44:45  39.976016  116.310973  1440   10
2101264  2008-06-17 09:44:46  39.975973  116.311003  1440   10
2101265  2008-06-17 09:44:47  39.975938  116.311025  1440   10
2101266  2008-06-17 09:44:48  39.975906  116.311038  1440   10
18592


In [23]:
pedestrians["tid"] += original.tid.max()
bikes["tid"] += (original.tid.max() + pedestrians.tid.max())
cars["tid"] += (original.tid.max() + pedestrians.tid.max() + bikes.tid.max())

In [24]:
augmented = pd.concat([pedestrians, bikes, cars, original], axis=0, ignore_index=True)
augmented.to_csv(data_path + "augmented_dataset.csv")